In [16]:
import os

import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from trl import DPOConfig, DPOTrainer
from typing import Dict

In [17]:
model_path = "/home/ltnga/ITDSIU21079/src/qwen_v2/checkpoint-100"
model = AutoModelForCausalLM.from_pretrained(model_path)
ref_model = AutoModelForCausalLM.from_pretrained(model_path)


tokenizer_path = "/home/ltnga/ITDSIU21079/src/qwen_v2/checkpoint-100"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  5.87it/s]


In [18]:
config = LoraConfig(
    r=32,
    lora_alpha=32,   
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",
)

In [19]:
model = get_peft_model(model, config)

In [50]:
print(dataset)

Dataset({
    features: ['chosen', 'rejected', 'conversations'],
    num_rows: 43904
})


In [51]:
dataset = dataset.filter(lambda x: x.get("chosen") and x.get("rejected"))


Filter: 100%|██████████| 43904/43904 [00:01<00:00, 23958.17 examples/s]


In [52]:
print(dataset[0]["chosen"])
print(dataset[0]["rejected"])


{'from': 'gpt', 'value': 'Người ngoại trái gió Zephyrian có khả năng nhận biết và diễn giải mẫu màu sắc và âm thanh một cách sâu sắc và liên quan chặt chẽ đến cấu trúc sinh học và xử lý thần kinh của họ. Từ góc độ sinh lý học, những sinh vật này đã tiến hóa để nhận biết một phổ màu sắc và sóng âm rộng hơn so với con người. Mắt của họ có thể phát hiện tần số ánh sáng mà con người không thể, dẫn đến khả năng nhìn thấy và diễn giải loạt màu sắc mà họ sử dụng để giao tiếp.\n\nHệ thống thính giác của họ, ngược lại, được điều chỉnh để phát hiện một phạm vi tần số rộng hơn, cho phép họ nhận biết và tạo ra những giai điệu mà chúng ta có thể thấy là kim loại hoặc kêu rát nhưng lại là phương tiện mạnh mẽ cho việc giao tiếp trong xã hội của họ. Ngoài ra, họ sở hữu một mạng thần kinh tiên tiến được thiết kế để giải mã những kết hợp phức tạp của màu sắc và âm thanh này, dịch chúng thành một hiểu biết toàn diện về cảm xúc, ý tưởng và ý định.\n\nVề việc diễn giải những mẫu màu sắc và âm thanh này, ng

In [54]:
def extract_prompt(example):
    chosen = example.get("chosen", "")
    rejected = example.get("rejected", "")
    
    if isinstance(chosen, str) and isinstance(rejected, str):
        for idx in range(min(len(chosen), len(rejected))):
            if chosen[idx] != rejected[idx]:
                if chosen[idx - 1] == " ":  # Remove space before the prompt
                    idx -= 1
                return {"chosen": chosen, "rejected": rejected[:idx]}
    
    # If data is invalid, return the example unchanged
    return example


In [55]:
train_dataset = dataset.map(
    extract_prompt,
    num_proc=training_args.dataset_num_proc,
    desc="Extracting prompts from dataset"
)


Extracting prompts from dataset: 100%|██████████| 43904/43904 [00:07<00:00, 5736.15 examples/s]


In [56]:
training_args = DPOConfig(output_dir="Qwen2.5-DPO", logging_steps=10)
trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)
trainer.train()

Extracting prompt from train dataset:   0%|          | 0/43904 [00:00<?, ? examples/s]


KeyError: 0